# Johns Hopkins - Data Science Specialization

## Programming Assignment 1

### Data

The zip file containing the data can be downloaded here:

    specdata.zip [2.4MB]

The zip file contains 332 comma-separated-value (CSV) files containing pollution monitoring data for fine particulate matter (PM) air pollution at 332 locations in the United States. Each file contains data from a single monitor and the ID number for each monitor is contained in the file name. For example, data for monitor 200 is contained in the file "200.csv". Each file contains three variables:

    Date: the date of the observation in YYYY-MM-DD format (year-month-day)
    sulfate: the level of sulfate PM in the air on that date (measured in micrograms per cubic meter)
    nitrate: the level of nitrate PM in the air on that date (measured in micrograms per cubic meter)

For this programming assignment you will need to unzip this file and create the directory 'specdata'. Once you have unzipped the zip file, do not make any modifications to the files in the 'specdata' directory. In each file you'll notice that there are many days where either sulfate or nitrate (or both) are missing (coded as NA). This is common with air pollution monitoring data in the United States.

### Part 1

Write a function named 'pollutantmean' that calculates the mean of a pollutant (sulfate or nitrate) across a specified list of monitors. The function 'pollutantmean' takes three arguments: 'directory', 'pollutant', and 'id'. Given a vector monitor ID numbers, 'pollutantmean' reads that monitors' particulate matter data from the directory specified in the 'directory' argument and returns the mean of the pollutant across all of the monitors, ignoring any missing values coded as NA.

In [52]:
pollutantmean <- function(directory, pollutant, id = 1:332){
  ## Return the mean of the pollutant across all monitors listed
  ## in the 'id' vector (ignoring NA values)
  df_colClasses = c("Date","double","double","integer")
  df_colNames = c("Date","sulfate","nitrate","ID")
  total_df <- read.table(text="",colClasses = df_colClasses,col.names = df_colNames)
  for(x in id)
  {
    file_name = paste(getwd(),"/",directory,"/", sprintf("%03d",x),".csv",sep="")
    new_df <- read.csv(file_name, header=TRUE)  
    total_df <- rbind(new_df, total_df)
  }
  poll_mean <- mean(total_df[[pollutant]], na.rm=TRUE)
  #print(total_df[1:2,])
  print(poll_mean)
}

### Comments on Solution

There are a few challenging things about this assignment:

* Understanding the nature of the data - that ultimately what you want is a single dataframe that contains all of the observations from all of the selected stations (IDs)
* Once you have that dataframe you will take the mean from the column as indicated by the
* Based on that you will need to iterate through the list of IDs which is passed to the function and then load the corresponding csv file
* Finding the corresponding csv file for the ID requires you to use the sprintf function to ensure that, not matter what the number is, it is represented by three digits. ie 1 => 001, 50 => 050, 121 => 121

In [2]:
setwd("C:\\Users\\harash\\Dropbox\\github-repo\\learning\\courses\\jhu-data-science-spec\\r-prog\\")
my_id <- c(1:3)
pollutantmean("specdata","sulfate",my_id)

[1] 4.389262


### Part 2

Write a function that reads a directory full of files and reports the number of completely observed cases in each data file. The function should return a data frame where the first column is the name of the file and the second column is the number of complete cases. A prototype of this function follows

In [53]:
row_complete <- function (row){
    for(val in row){
        if(is.na(val)){return(FALSE)}
        else if(val == "NA"){return(FALSE)}
    }
    return(TRUE)
}

count_complete_rows_df2 <- function(dataframe){
    total_complete_rows <- 0
    for(row in 1:nrow(dataframe)){
        if(sum(is.na(dataframe[row,])) == 0){
            total_complete_rows<-total_complete_rows+1
        }
    }
    return(total_complete_rows)
}

count_complete_rows_df <- function(dataframe){
    total_complete_rows <- 0
    for(row in 1:nrow(dataframe)){
        if(row_complete(dataframe[row,])){
            total_complete_rows<-total_complete_rows+1
        }
    }
    return(total_complete_rows)
}

complete <- function(directory, ids = 1:332){
  df_colClasses = c("character","integer")
  df_colNames = c("FileName","Count")
  total_df <- read.table(text="",colClasses = df_colClasses,col.names = df_colNames)
  for(id in ids){
      file_name = paste(getwd(),"/",directory,"/", sprintf("%03d",id),".csv",sep="")
      #print(file_name)
      df_colClasses = c("character","double","double","integer")
      new_df <- read.csv(file_name, header=TRUE, colClasses = df_colClasses) 
      #print(new_df[5,])
      #print(sum(is.na(new_df[5,])))
      nobs <- count_complete_rows_df(new_df)
      new_entry <- data.frame(id, nobs)
      total_df <- rbind(new_entry, total_df)
  }
    return(total_df)
}

In [4]:
complete("specdata", 1)
##   id nobs
## 1  1  117

id,nobs
1,117


In [5]:
complete("specdata", c(2, 4, 8, 10, 12))
##   id nobs
## 1  2 1041
## 2  4  474
## 3  8  192
## 4 10  148
## 5 12   96

id,nobs
12,96
10,148
8,192
4,474
2,1041


In [6]:
complete("specdata", 30:25)
##   id nobs
## 1 30  932
## 2 29  711
## 3 28  475
## 4 27  338
## 5 26  586
## 6 25  463

id,nobs
25,463
26,586
27,338
28,475
29,711
30,932


### Part 3

Write a function that takes a directory of data files and a threshold for complete cases and calculates the correlation between sulfate and nitrate for monitor locations where the number of completely observed cases (on all variables) is greater than the threshold. The function should return a vector of correlations for the monitors that meet the threshold requirement. If no monitors meet the threshold requirement, then the function should return a numeric vector of length 0. A prototype of this function follows

In [68]:
corr <- function(directory, threshold = 0){
    
    all_corr <- vector('numeric')
    complete_df <- complete(directory)
    for(row in 1:nrow(complete_df)){
        if(complete_df[row, "nobs"] > threshold){
            id_corr <- file_correlation(row, directory)
            all_corr <- c(all_corr, id_corr)
        }
    }
    return(all_corr)
}

file_correlation <- function(id, directory){

    file_name = paste(getwd(),"/",directory,"/", sprintf("%03d",id),".csv",sep="")
    df_colClasses = c("character","double","double","integer")
    new_df <- read.csv(file_name, header=TRUE, colClasses = df_colClasses) 
    #print(new_df[,1:2])
    id_corr <- cor(new_df$sulfate,new_df$nitrate,use="pairwise.complete.obs")
    return(id_corr)
}

In [69]:
cr <- corr("specdata", 150)
head(cr)
## [1] -0.01895754 -0.14051254 -0.04389737 -0.06815956 -0.12350667 -0.07588814
#file_correlation(1,"specdata")

[1] -0.01895754 -0.14051254 -0.04389737 -0.06815956 -0.12350667 -0.07588814

In [49]:
summary(cr)
##     Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
## -0.21057 -0.04999  0.09463  0.12525  0.26844  0.76313

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
-1.00000 -0.06437  0.06476  0.11682  0.25894  1.00000        7 

In [50]:
cr <- corr("specdata", 400)
head(cr)
## [1] -0.01895754 -0.04389737 -0.06815956 -0.07588814  0.76312884 -0.15782860


[1] -0.14051254 -0.04389737 -0.06815956 -0.15967365 -0.54537269 -0.15699892

In [51]:
summary(cr)
##     Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
## -0.17623 -0.03109  0.10021  0.13969  0.26849  0.76313

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
-1.00000 -0.06797  0.04161  0.10744  0.23312  1.00000        3 

## Prototyping Section

In [46]:
#file_name <- "C:\\Users\\harash\\Dropbox\\github-repo\\learning\\courses\\jhu-data-science-spec\\r-prog\\specdata\\001.csv"
#new_df <- read.csv(file_name, header=TRUE) 
#new_df[,2]
#id_corr <- cor(new_df$sulfate,new_df$nitrate, use='complete.obs')
#print(id_corr)
corr <- vector('numeric')
corr <- c(corr,-.5490)
corr <- c(corr, 1.5)
print(corr)

[1] -0.549  1.500


In [36]:
complete_df <- complete("specdata")
complete_df[3, "nobs"]

[1] 447

In [ ]:
is.na(new_df)

In [12]:
list(paste(sprintf("%03d",2),".csv",sep=""))

[[1]]
[1] "002.csv"

In [11]:
list(1,2,3,4))

[[1]]
[1] 1

[[2]]
[1] 2

[[3]]
[1] 3

[[4]]
[1] 4



In [56]:
pollutantmean("specdata", "sulfate", 1:10)

[1] 4.064128


In [57]:
pollutantmean("specdata", "nitrate", 70:72)

[1] 1.706047


In [58]:
pollutantmean("specdata", "sulfate", 34)

[1] 1.477143


In [59]:
pollutantmean("specdata", "nitrate")

[1] 1.702932


In [60]:
cc <- complete("specdata", c(6, 10, 20, 34, 100, 200, 310))
print(cc$nobs)

[1] 232 460 104 165 124 148 228


In [61]:
cc <- complete("specdata", 54)
print(cc$nobs)

[1] 219


In [63]:
set.seed(42)
cc <- complete("specdata", 332:1)
use <- sample(332, 10)
print(cc[use, "nobs"])

 [1] 135  65  81   4 121 741 439 283 345 548


In [70]:
cr <- corr("specdata")                
cr <- sort(cr)                
set.seed(868)                
out <- round(cr[sample(length(cr), 5)], 4)
print(out)

[1]  0.2621  0.1072 -0.0088  0.4407  0.0419


In [71]:
cr <- corr("specdata", 129)                
cr <- sort(cr)                
n <- length(cr)                
set.seed(197)                
out <- c(n, round(cr[sample(n, 5)], 4))
print(out)

[1] 236.0000   0.2481   0.0413  -0.1563  -0.1781   0.7173


In [72]:
cr <- corr("specdata", 2000)                
n <- length(cr)                
cr <- corr("specdata", 1000)                
cr <- sort(cr)
print(c(n, round(cr, 4)))

[1] 0.0000 0.0229 0.2365 0.2871
